# Hackathon Dotz

### Importando Bibliotecas

In [0]:
# Importação dos pacotes para Analise

import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats as sct
import statsmodels.api as sm
import re
import tensorflow as tf
import torch
import tqdm
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization
from tensorflow.keras.layers import Embedding
from tensorflow.keras import layers
from itertools import chain
from google.colab import files  


### Carregando Arquivos

In [14]:
#Carregar os dados para o google colab
uploaded = files.upload()

Saving Dados Gerais Datasets.txt to Dados Gerais Datasets.txt


In [0]:
# Importação dos dataset

df = pd.read_csv('Hackathon_Base_Treino_comdep.csv')
df2 = pd.read_csv( 'Hackathon_Base_Teste.csv')

# Etapa 1 

### Processamento de Dados

In [0]:
# Excluindo da descrição texto após os números, informações julgadas irrelevantes para a classificação.
df['DESC_AJUSTADA'] = df['DESCRIÇÃO PARCEIRO'].str.lower().str.replace('[0-9].+', '', regex=True)

# Excluindo da descrição ajustada as stop_words, informações julgadas irrelevantes para a classificação.
stop_words=['de','da','do','com','para','c/','kg','un','ml','pct','gfa','jg','p/','la','&','.',',','-','+']
for x in stop_words:
  df['DESC_AJUSTADA'] = df['DESC_AJUSTADA'].str.replace(x, '')

In [17]:
# Organizando as DEPARTAMENTO
samples = []
labels = []
class_names = []
prod_index = 0
departamento = df[['DEPARTAMENTO','DESC_AJUSTADA']].groupby(['DEPARTAMENTO']).agg(['count'])
departamento.columns = ['count']
departamento.sort_values('count', ascending=False, inplace=True)
departamento = list(departamento.index)
for dep in departamento:
    class_names.append(dep)
    produtos = len(df[df.DEPARTAMENTO == dep])
    print("Processing %s, %d products found" % (dep, produtos))
    produto = df[df.DEPARTAMENTO == dep]['DESC_AJUSTADA'].str.strip()
    #produto = list(produto.str.split().array)
    #produto = list(chain(*produto))
    #produto = sorted(set(produto))
    for prod in produto:
        samples.append(prod)
        labels.append(prod_index)    
    prod_index += 1

print("\nClasses:", class_names)
print("\nNumber of samples:", len(samples))
print("\nNumber of labels:", len(labels))

Processing MERCEARIA DOCE, 3614 products found
Processing MERCEARIA SALGADA, 2090 products found
Processing UTILIDADES DE COZINHA, 1670 products found
Processing BEBIDAS ALCOÓLICAS, 1366 products found
Processing CUIDADOS COM O CABELO, 1257 products found
Processing BEBIDAS NÃO ALCOÓLICAS, 698 products found
Processing CUIDADOS COM O CORPO, 658 products found
Processing HORTIFRUTI, 642 products found
Processing CALÇADO, 607 products found
Processing DESCARTÁVEIS, 598 products found
Processing FRIOS, 493 products found
Processing CAMA, MESA E BANHO, 453 products found
Processing CUIDADOS COM A CASA, 449 products found
Processing LAVANDERIA, 448 products found
Processing AÇOUGUE, 420 products found
Processing PAPELARIA, 387 products found
Processing IOGURTES, 360 products found
Processing ELETRO, 356 products found
Processing PET SHOP, 343 products found
Processing HIGIENE PESSOAL, 339 products found
Processing CONGELADOS DOCES, 293 products found
Processing DECORAÇÃO, 288 products found

### Dividindo Treinamento e Teste

In [0]:
# Shuffle the data
seed = 1337
rng = np.random.RandomState(seed)
rng.shuffle(samples)
rng = np.random.RandomState(seed)
rng.shuffle(labels)

# Extract a training & validation split
validation_split = 0.1
num_validation_samples = int(validation_split * len(samples))
train_samples = samples[:-num_validation_samples]
val_samples = samples[-num_validation_samples:]
train_labels = labels[:-num_validation_samples]
val_labels = labels[-num_validation_samples:]

### Crianção do vocabulário

In [0]:
vectorizer = TextVectorization(standardize='lower_and_strip_punctuation')
text_ds = tf.data.Dataset.from_tensor_slices(train_samples).batch(128)
vectorizer.adapt(text_ds)

voc = samples
word_index = dict(zip(voc, range(2, len(voc))))

### Carregar palavras pré-treinadas

In [20]:
palavras = sorted(set(samples))
embeddings_index = {item: np.random.random_sample(100) for item in tqdm.notebook.tqdm(palavras)}

print("Found %s word vectors." % len(embeddings_index))


Found 19670 word vectors.


In [21]:
num_tokens = len(voc) + 2
embedding_dim = 100
hits = 0
misses = 0

# Prepare embedding matrix
embedding_matrix = np.zeros((num_tokens, embedding_dim))
for word, i in word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # Words not found in embedding index will be all-zeros.
        # This includes the representation for "padding" and "OOV"
        embedding_matrix[i] = embedding_vector
        hits += 1
    else:
        misses += 1
print("Converted %d words (%d misses)" % (hits, misses))

Converted 19668 words (0 misses)


In [0]:
embedding_layer = Embedding(
    num_tokens,
    embedding_dim,
    embeddings_initializer=keras.initializers.Constant(embedding_matrix),
    trainable=False,
)

### Construindo o Modelo

In [23]:
int_sequences_input = keras.Input(shape=(None,), dtype="int64")
embedded_sequences = embedding_layer(int_sequences_input)
x = layers.Conv1D(128, 1, activation="relu")(embedded_sequences)
x = layers.MaxPooling1D(1)(x)
x = layers.Conv1D(128, 1, activation="relu")(x)
x = layers.MaxPooling1D(1)(x)
x = layers.Conv1D(128, 1, activation="relu")(x)
x = layers.GlobalMaxPooling1D()(x)
x = layers.Dense(128, activation="relu")(x)
x = layers.Dropout(0.2)(x)
preds = layers.Dense(len(class_names), activation="softmax")(x)
model = keras.Model(int_sequences_input, preds)
model.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, None)]            0         
_________________________________________________________________
embedding_1 (Embedding)      (None, None, 100)         2201100   
_________________________________________________________________
conv1d_3 (Conv1D)            (None, None, 128)         12928     
_________________________________________________________________
max_pooling1d_2 (MaxPooling1 (None, None, 128)         0         
_________________________________________________________________
conv1d_4 (Conv1D)            (None, None, 128)         16512     
_________________________________________________________________
max_pooling1d_3 (MaxPooling1 (None, None, 128)         0         
_________________________________________________________________
conv1d_5 (Conv1D)            (None, None, 128)         1651

### Treinando o Modelo

In [31]:
y_train


array([ 4, 22,  2, ..., 21, 12,  2])

In [0]:
x_train = vectorizer(np.array([[s] for s in train_samples])).numpy()
x_val = vectorizer(np.array([[s] for s in val_samples])).numpy()

y_train = np.array(train_labels)
y_val = np.array(val_labels)

In [25]:
batch_size = 128
epochs = 70
model.compile(loss="sparse_categorical_crossentropy", optimizer="rmsprop", metrics=["accuracy"])
model.fit(x_train, y_train, batch_size=batch_size, epochs=epochs, validation_data=(x_val, y_val))

Epoch 1/70
155/155 [==============================] - 1s 6ms/step - loss: 3.0640 - accuracy: 0.2464 - val_loss: 2.6748 - val_accuracy: 0.3464
Epoch 2/70
155/155 [==============================] - 1s 4ms/step - loss: 2.4384 - accuracy: 0.4034 - val_loss: 2.2148 - val_accuracy: 0.4841
Epoch 3/70
155/155 [==============================] - 1s 4ms/step - loss: 2.0402 - accuracy: 0.5040 - val_loss: 1.8162 - val_accuracy: 0.5618
Epoch 4/70
155/155 [==============================] - 1s 4ms/step - loss: 1.7478 - accuracy: 0.5768 - val_loss: 1.5928 - val_accuracy: 0.6250
Epoch 5/70
155/155 [==============================] - 1s 4ms/step - loss: 1.5329 - accuracy: 0.6245 - val_loss: 1.3909 - val_accuracy: 0.6550
Epoch 6/70
155/155 [==============================] - 1s 4ms/step - loss: 1.3556 - accuracy: 0.6677 - val_loss: 1.2606 - val_accuracy: 0.6909
Epoch 7/70
155/155 [==============================] - 1s 4ms/step - loss: 1.2183 - accuracy: 0.7000 - val_loss: 1.2002 - val_accuracy: 0.7064
Epoch 

KeyboardInterrupt: ignored

In [0]:
# Excluindo da descrição texto após os números, informações julgadas irrelevantes para a classificação.
df2['DESC_AJUSTADA'] = df2['DESCRIÇÃO PARCEIRO'].str.lower().str.replace('[0-9].+', '', regex=True)
# Excluindo da descrição ajustada as stop_words, informações julgadas irrelevantes para a classificação.
stop_words=['de','da','do','com','para','c/','kg','un','ml','pct','gfa','jg','p/','la','&','.',',','-','+']
for x in stop_words:
  df2['DESC_AJUSTADA'] = df2['DESC_AJUSTADA'].str.replace(x, '')

In [0]:
string_input = keras.Input(shape=(1,), dtype="string")
x = vectorizer(string_input)
preds = model(x)
end_to_end_model = keras.Model(string_input, preds)

exemplos = [[x] for x in df2['DESC_AJUSTADA']]
#exemplos = [['Exemplo']] #Informa DF
dep = []
for exemplo in tqdm.notebook.tqdm(exemplos):
  probabilities = end_to_end_model.predict([[exemplo]])
  dep.append(class_names[np.argmax(probabilities[0])])
  #print('DESCRIÇÃO: ', exemplo, '| DEPARTAMENTO: ', class_names[np.argmax(probabilities[0])])

In [0]:
# Ajustando dados de testes com as previsões
df2['DEPARTAMENTO'] = dep
df2.head(2)

# Etapa 2

In [0]:
# Excluindo da descrição texto após os números, informações julgadas irrelevantes para a classificação.
df['DESC_AJUSTADA'] = df['DESCRIÇÃO PARCEIRO'].str.lower().str.replace('[0-9].+', '', regex=True)

# Excluindo da descrição ajustada as stop_words, informações julgadas irrelevantes para a classificação.
stop_words=['de','da','do','com','para','c/','kg','un','ml','pct','gfa','jg','p/','la','&','.',',','-','+']
for x in stop_words:
  df['DESC_AJUSTADA'] = df['DESC_AJUSTADA'].str.replace(x, '')

# Incluindo o DEPARTAMENTO para Otimizar a performance
df['DESC_AJUSTADA'] = df['DEPARTAMENTO'].str.lower() +' '+ df['DESC_AJUSTADA']

In [0]:
# Organizando as CATEGORIA
samples = []
labels = []
class_names = []
prod_index = 0
categoria = df[['CATEGORIA','DESC_AJUSTADA']].groupby(['CATEGORIA']).agg(['count'])
categoria.columns = ['count']
categoria.sort_values('count', ascending=False, inplace=True)
categoria = list(categoria.index)
for cat in categoria:
    class_names.append(cat)
    produtos = len(df[df.CATEGORIA == cat])
    print("Processing %s, %d products found" % (cat, produtos))
    produto = df[df.CATEGORIA == cat]['DESC_AJUSTADA'].str.strip()
    #produto = list(produto.str.split().array)
    #produto = list(chain(*produto))
    #produto = sorted(set(produto))
    for prod in produto:
        samples.append(prod)
        labels.append(prod_index)    
    prod_index += 1

print("\nClasses:", class_names)
print("\nNumber of samples:", len(samples))
print("\nNumber of labels:", len(labels))

In [0]:
# Shuffle the data
seed = 1337
rng = np.random.RandomState(seed)
rng.shuffle(samples)
rng = np.random.RandomState(seed)
rng.shuffle(labels)

# Extract a training & validation split
validation_split = 0.1
num_validation_samples = int(validation_split * len(samples))
train_samples = samples[:-num_validation_samples]
val_samples = samples[-num_validation_samples:]
train_labels = labels[:-num_validation_samples]
val_labels = labels[-num_validation_samples:]

In [0]:
vectorizer = TextVectorization(standardize='lower_and_strip_punctuation')
text_ds = tf.data.Dataset.from_tensor_slices(train_samples).batch(128)
vectorizer.adapt(text_ds)

voc = samples
word_index = dict(zip(voc, range(2, len(voc))))

In [0]:
palavras = sorted(set(samples))
embeddings_index = {item: np.random.random_sample(100) for item in tqdm.notebook.tqdm(palavras)}

print("Found %s word vectors." % len(embeddings_index))

In [0]:
num_tokens = len(voc) + 2
embedding_dim = 100
hits = 0
misses = 0

# Prepare embedding matrix
embedding_matrix = np.zeros((num_tokens, embedding_dim))
for word, i in word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # Words not found in embedding index will be all-zeros.
        # This includes the representation for "padding" and "OOV"
        embedding_matrix[i] = embedding_vector
        hits += 1
    else:
        misses += 1
print("Converted %d words (%d misses)" % (hits, misses))

In [0]:
embedding_layer = Embedding(
    num_tokens,
    embedding_dim,
    embeddings_initializer=keras.initializers.Constant(embedding_matrix),
    trainable=False,
)

In [0]:
int_sequences_input = keras.Input(shape=(None,), dtype="int64")
embedded_sequences = embedding_layer(int_sequences_input)
x = layers.Conv1D(128, 1, activation="relu")(embedded_sequences)
x = layers.MaxPooling1D(1)(x)
x = layers.Conv1D(128, 1, activation="relu")(x)
x = layers.MaxPooling1D(1)(x)
x = layers.Conv1D(128, 1, activation="relu")(x)
x = layers.GlobalMaxPooling1D()(x)
x = layers.Dense(128, activation="relu")(x)
x = layers.Dropout(0.2)(x)
preds = layers.Dense(len(class_names), activation="softmax")(x)
model = keras.Model(int_sequences_input, preds)
model.summary()

In [0]:
x_train = vectorizer(np.array([[s] for s in train_samples])).numpy()
x_val = vectorizer(np.array([[s] for s in val_samples])).numpy()

y_train = np.array(train_labels)
y_val = np.array(val_labels)

In [0]:
batch_size = 128
epochs = 70
model.compile(loss="sparse_categorical_crossentropy", optimizer="rmsprop", metrics=["accuracy"])
model.fit(x_train, y_train, batch_size=batch_size, epochs=epochs, validation_data=(x_val, y_val))

In [0]:
# Incluindo o DEPARTAMENTO para Otimizar a performance
df2['DESC_AJUSTADA'] = df2['DEPARTAMENTO'].str.lower() +' '+ df['DESC_AJUSTADA']

In [0]:
string_input = keras.Input(shape=(1,), dtype="string")
x = vectorizer(string_input)
preds = model(x)
end_to_end_model = keras.Model(string_input, preds)

exemplos = [[x] for x in df2['DESC_AJUSTADA']]
#exemplos = [['Exemplo']] #Informa DF
cat = []
for exemplo in tqdm.notebook.tqdm(exemplos):
  probabilities = end_to_end_model.predict([[exemplo]])
  cat.append(class_names[np.argmax(probabilities[0])])
  #print('DESCRIÇÃO: ', exemplo, '| CATEGORIA: ', class_names[np.argmax(probabilities[0])])

In [0]:
# Ajustando dados de testes com as previsões
df2['CATEGORIA'] = cat
df2.head(2)

# Etapa 3

In [0]:
# Ajustando nome da coluna, para facilitar o código por causa do -
df.rename(columns={'SUB-CATEGORIA': 'SUBCATEGORIA'}, inplace=True)

In [0]:
# Excluindo da descrição texto após os números, informações julgadas irrelevantes para a classificação.
df['DESC_AJUSTADA'] = df['DESCRIÇÃO PARCEIRO'].str.lower()

# Excluindo da descrição ajustada as stop_words, informações julgadas irrelevantes para a classificação.
stop_words=['de','da','do','com','para','c/','kg','un','ml','pct','gfa','jg','p/','la','&','.',',','-','+']
for x in stop_words:
  df['DESC_AJUSTADA'] = df['DESC_AJUSTADA'].str.replace(x, '')

# Incluindo o DEPARTAMENTO e CATEGORIA para Otimizar a performance
df['DESC_AJUSTADA'] = df['DEPARTAMENTO'].str.lower() +' '+ df['CATEGORIA'].str.lower() +' '+ df['DESC_AJUSTADA']

In [0]:
# Organizando as SUB-CATEGORIA
samples = []
labels = []
class_names = []
prod_index = 0
subcategoria = df[['SUBCATEGORIA','DESC_AJUSTADA']].groupby(['SUBCATEGORIA']).agg(['count'])
subcategoria.columns = ['count']
subcategoria.sort_values('count', ascending=False, inplace=True)
subcategoria = list(subcategoria.index)
for sub in subcategoria:
    class_names.append(sub)
    produtos = len(df[df.SUBCATEGORIA == sub])
    print("Processing %s, %d products found" % (sub, produtos))
    produto = df[df.SUBCATEGORIA == sub]['DESC_AJUSTADA'].str.strip()
    #produto = list(produto.str.split().array)
    #produto = list(chain(*produto))
    #produto = sorted(set(produto))
    for prod in produto:
        samples.append(prod)
        labels.append(prod_index)    
    prod_index += 1

print("\nClasses:", class_names)
print("\nNumber of samples:", len(samples))
print("\nNumber of labels:", len(labels))

In [0]:
# Shuffle the data
seed = 1337
rng = np.random.RandomState(seed)
rng.shuffle(samples)
rng = np.random.RandomState(seed)
rng.shuffle(labels)

# Extract a training & validation split
validation_split = 0.1
num_validation_samples = int(validation_split * len(samples))
train_samples = samples[:-num_validation_samples]
val_samples = samples[-num_validation_samples:]
train_labels = labels[:-num_validation_samples]
val_labels = labels[-num_validation_samples:]

In [0]:
vectorizer = TextVectorization(standardize='lower_and_strip_punctuation')
text_ds = tf.data.Dataset.from_tensor_slices(train_samples).batch(128)
vectorizer.adapt(text_ds)

voc = samples
word_index = dict(zip(voc, range(2, len(voc))))

In [0]:

palavras = sorted(set(samples))
embeddings_index = {item: np.random.random_sample(100) for item in tqdm.notebook.tqdm(palavras)}

print("Found %s word vectors." % len(embeddings_index))

In [0]:
num_tokens = len(voc) + 2
embedding_dim = 100
hits = 0
misses = 0

# Prepare embedding matrix
embedding_matrix = np.zeros((num_tokens, embedding_dim))
for word, i in word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # Words not found in embedding index will be all-zeros.
        # This includes the representation for "padding" and "OOV"
        embedding_matrix[i] = embedding_vector
        hits += 1
    else:
        misses += 1
print("Converted %d words (%d misses)" % (hits, misses))

In [0]:
embedding_layer = Embedding(
    num_tokens,
    embedding_dim,
    embeddings_initializer=keras.initializers.Constant(embedding_matrix),
    trainable=False,
)

In [0]:
int_sequences_input = keras.Input(shape=(None,), dtype="int64")
embedded_sequences = embedding_layer(int_sequences_input)
x = layers.Conv1D(128, 1, activation="relu")(embedded_sequences)
x = layers.MaxPooling1D(1)(x)
x = layers.Conv1D(128, 1, activation="relu")(x)
x = layers.MaxPooling1D(1)(x)
x = layers.Conv1D(128, 1, activation="relu")(x)
x = layers.GlobalMaxPooling1D()(x)
x = layers.Dense(128, activation="relu")(x)
x = layers.Dropout(0.2)(x)
preds = layers.Dense(len(class_names), activation="softmax")(x)
model = keras.Model(int_sequences_input, preds)
model.summary()


In [0]:
x_train = vectorizer(np.array([[s] for s in train_samples])).numpy()
x_val = vectorizer(np.array([[s] for s in val_samples])).numpy()

y_train = np.array(train_labels)
y_val = np.array(val_labels)

In [0]:
batch_size = 128
epochs = 120
model.compile(loss="sparse_categorical_crossentropy", optimizer="rmsprop", metrics=["accuracy"])
model.fit(x_train, y_train, batch_size=batch_size, epochs=epochs, validation_data=(x_val, y_val))

In [0]:
# Ajustando descrição.
df2['DESC_AJUSTADA'] = df2['DESCRIÇÃO PARCEIRO'].str.lower()
# Excluindo da descrição ajustada as stop_words, informações julgadas irrelevantes para a classificação.
stop_words=['de','da','do','com','para','c/','kg','un','ml','pct','gfa','jg','p/','la','&','.',',','-','+']
for x in stop_words:
  df2['DESC_AJUSTADA'] = df2['DESC_AJUSTADA'].str.replace(x, '')

# Incluindo o DEPARTAMENTO e CATEGORIA para Otimizar a performance
df2['DESC_AJUSTADA'] = df2['DEPARTAMENTO'].str.lower() +' '+ df2['CATEGORIA'].str.lower() +' '+ df2['DESC_AJUSTADA']

In [0]:
string_input = keras.Input(shape=(1,), dtype="string")
x = vectorizer(string_input)
preds = model(x)
end_to_end_model = keras.Model(string_input, preds)

exemplos = [[x] for x in df2['DESC_AJUSTADA']]
#exemplos = [['Exemplo']] #Informa DF
sub = []
for exemplo in tqdm.notebook.tqdm(exemplos):
  probabilities = end_to_end_model.predict([[exemplo]])
  sub.append(class_names[np.argmax(probabilities[0])])
  #print('DESCRIÇÃO: ', exemplo, '| DEPARTAMENTO: ', class_names[np.argmax(probabilities[0])])

In [0]:
# Ajustando dados de testes com as previsões
df2['SUB-CATEGORIA'] = sub
df2.head(2)

In [0]:
dotz = df2[['DESCRIÇÃO PARCEIRO', 'SUB-CATEGORIA', 'CATEGORIA']]
dotz.to_csv('dotz.csv')